In [45]:
pip install pandas numpy scikit-learn flask streamlit

In [47]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [49]:
file_path = 'c:/Users/suwat/Downloads/BooksDatasetClean.csv'
df = pd.read_csv(file_path, on_bad_lines='skip')

In [50]:
df.head()

,Title,Authors,Description,Category,Publisher,Price Starting With ($),Publish Date (Month),Publish Date (Year)
0,Goat Brothers,"By Colton, Larry",NaN,"History , General",Doubleday,8.79,January,1993
1,The Missing Person,"By Grumbach, Doris",NaN,"Fiction , General",Putnam Pub Group,4.99,March,1981
2,Don't Eat Your Heart Out Cookbook,"By Piscatella, Joseph C.",NaN,"Cooking , Reference",Workman Pub Co,4.99,September,1983
3,When Your Corporate Umbrella Begins to Leak: A...,"By Davis, Paul D.",NaN,NaN,Natl Pr Books,4.99,April,1991
4,Amy Spangler's Breastfeeding : A Parent's Guide,"By Spangler, Amy",NaN,NaN,Amy Spangler,5.32,February,1997


In [51]:
df.dropna(subset=['Description'], inplace=True)  # Remove rows with missing description

In [52]:
#Vectorize the descriptions using TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Description'])


In [53]:
# Convert the matrix to a sparse matrix of type float32 for memory efficiency
tfidf_matrix_sparse = tfidf_matrix.astype('float32')


In [ ]:
# Apply approximate Nearest Neighbors (ANN) for efficient similarity search
knn = NearestNeighbors(n_neighbors=6, metric='cosine', n_jobs=-1)  # n_jobs=-1 uses all CPU cores
knn.fit(tfidf_matrix_sparse)  # Fit the model with the TF-IDF matrix


NearestNeighbors(metric='cosine', n_jobs=-1, n_neighbors=6)

In [55]:
# define the recommendation function
def get_recommendations_ann(book_title, df, knn):
    # Find the index of the book
    idx = df[df['Title'] == book_title].index[0]
    
    # Get the vector representation of the book's description
    book_vector = tfidf_matrix[idx]
    
    # Find the nearest neighbors (most similar books)
    distances, indices = knn.kneighbors(book_vector, n_neighbors=6)  # n_neighbors=6 to include the input book
    
    # Get the recommended book titles (skip the first one as it will be the input book itself)
    recommended_books = df['Title'].iloc[indices[0][1:]]  # Exclude the input book itself
    
    return recommended_books

In [ ]:
def get_recommendations_ann(book_title, df, knn):
    # Check if the book exists in the dataset
    if book_title not in df['Title'].values:
        return f"Sorry, the book '{book_title}' was not found in the dataset."
    
    # Find the index of the book
    idx = df[df['Title'] == book_title].index[0]
    
    # Get the vector representation of the book's description
    book_vector = tfidf_matrix[idx]
    
    # Find the nearest neighbors (most similar books)
    distances, indices = knn.kneighbors(book_vector, n_neighbors=6)  # n_neighbors=6 to include the input book
    
    # Get the recommended book titles (skip the first one as it will be the input book itself)
    recommended_books = df['Title'].iloc[indices[0][1:]]  # Exclude the input book itself
    
    return recommended_books

# Test the function with a book title
book_title = 'American Dreams: Lost & Found'  
recommendations = get_recommendations_ann(book_title, df, knn)

# Display the recommendations or the error message
if isinstance(recommendations, str):
    print(recommendations)  # If it's an error message
else:
    print("Recommended books based on '{}':".format(book_title))
    for idx, book in enumerate(recommendations, start=1):
        print(f"{idx}. {book}")


Recommended books based on 'American Dreams: Lost & Found':
1. The Divide
2. Felix in the Underworld
3. Back To The Divide
4. A Stolen Tongue
5. Acheson Country: A Memoir
